In [1]:
import nltk
from snowballstemmer import TurkishStemmer
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import numpy
import random
import json


with open(r"C:\Users\user\Desktop\Zlik yap\data.json") as file:
    data = json.load(file)


stemmer=TurkishStemmer()
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stemWord(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stemWord(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

Using TensorFlow backend.


In [3]:
labels

['ayrılma', 'bilgi', 'learn', 'selamlama']

In [2]:
model = Sequential()
model.add(Dense(16,input_shape=(len(training[0]),),activation="relu"))
model.add(Dense(16,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(4,activation="softmax"))
model.summary()
model.compile(Adam(lr=.001),loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(training, output,epochs=300, verbose=2,batch_size=4)

W0309 12:27:23.517306  1588 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0309 12:27:23.549221  1588 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0309 12:27:23.556203  1588 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0309 12:27:23.591112  1588 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\envs\gputest\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                576       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 916
Trainable params: 916
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
 - 3s - loss: 1.4130 - acc: 0.1667
Epoch 2/300
 - 0s - loss: 1.3256 - acc: 0.2778
Epoch 3/300
 - 0s - loss: 1.3242 - acc: 0.2778
Epoch 4/300
 - 0s - loss: 1.3617 - acc: 0.2222
Epoch 5/300
 - 0s - loss: 1.3382 - acc: 0.2778
Epoch 6/300
 - 0s - loss: 1.3089 - acc: 0

Epoch 153/300
 - 0s - loss: 0.0743 - acc: 1.0000
Epoch 154/300
 - 0s - loss: 0.0928 - acc: 1.0000
Epoch 155/300
 - 0s - loss: 0.0938 - acc: 1.0000
Epoch 156/300
 - 0s - loss: 0.1014 - acc: 1.0000
Epoch 157/300
 - 0s - loss: 0.1180 - acc: 0.9444
Epoch 158/300
 - 0s - loss: 0.0787 - acc: 1.0000
Epoch 159/300
 - 0s - loss: 0.1180 - acc: 0.9444
Epoch 160/300
 - 0s - loss: 0.0892 - acc: 1.0000
Epoch 161/300
 - 0s - loss: 0.1748 - acc: 1.0000
Epoch 162/300
 - 0s - loss: 0.0605 - acc: 1.0000
Epoch 163/300
 - 0s - loss: 0.1032 - acc: 1.0000
Epoch 164/300
 - 0s - loss: 0.1705 - acc: 0.9444
Epoch 165/300
 - 0s - loss: 0.1020 - acc: 1.0000
Epoch 166/300
 - 0s - loss: 0.0504 - acc: 1.0000
Epoch 167/300
 - 0s - loss: 0.0627 - acc: 1.0000
Epoch 168/300
 - 0s - loss: 0.1199 - acc: 0.9444
Epoch 169/300
 - 0s - loss: 0.1323 - acc: 0.9444
Epoch 170/300
 - 0s - loss: 0.0693 - acc: 1.0000
Epoch 171/300
 - 0s - loss: 0.1374 - acc: 0.9444
Epoch 172/300
 - 0s - loss: 0.0318 - acc: 1.0000
Epoch 173/300
 - 0s 

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stemWord(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Chatbot ile konuşmaya başlayabilirsiniz (quit yazarak çıkabilirsiniz)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(np.asanyarray([bag_of_words(inp, words)]))[0]
        print(results)
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.85:
            
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            print("Tam olarak anlayamadım")

chat()

Chatbot ile konuşmaya başlayabilirsiniz (quit yazarak çıkabilirsiniz)!
You: selam chat bot
[1.0510883e-03 1.3894166e-03 5.9989234e-04 9.9695957e-01]
Nasıl yardımcı olabilirim
You: yapay zekadan bahseder misin merak ediyorum
[4.6797752e-04 9.8351544e-01 8.3802072e-03 7.6363515e-03]
Yapay zekâ, bir bilgisayarın veya bilgisayar kontrolündeki bir robotun çeşitli faaliyetleri zeki canlılara benzer şekilde yerine getirme kabiliyeti
